In [1]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 56.4 MB/s  0:00:006m0:00:01


In [8]:
%pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 54.5 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pandas]2m1/2 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import duckdb
import os

conn = duckdb.connect("quickbite_express.db")

dataset_path = "/workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets"

In [2]:

for file in os.listdir(dataset_path):
    if file.endswith(".csv"):
        table_name = file.replace(".csv", "")
        file_path = os.path.join(dataset_path, file)

        conn.execute(f"""
            CREATE TABLE IF NOT EXISTS {table_name} AS
            SELECT * FROM '{file_path}';
        """)
        print(f"{table_name} table created using {file_path}")

fact_order_items table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/fact_order_items.csv
dim_customer table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/dim_customer.csv
dim_restaurant table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/dim_restaurant.csv
fact_ratings table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/fact_ratings.csv
fact_orders table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/fact_orders.csv
dim_menu_item table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/dim_menu_item.csv
fact_delivery_performance table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/fact_delivery_performance.csv
dim_delivery_partner_ table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/dim_delivery_partner_.csv


In [2]:
conn.execute("SHOW TABLES;").fetch_df()

,name
0,dim_customer
1,dim_delivery_partner_
2,dim_menu_item
3,dim_restaurant
4,fact_delivery_performance
5,fact_order_items
6,fact_orders
7,fact_ratings


In [3]:
fact_orders_df = conn.execute("SELECT * FROM fact_orders;").fetch_df()

fact_orders_df.head()

,order_id,customer_id,restaurant_id,delivery_partner_id,order_timestamp,subtotal_amount,discount_amount,delivery_fee,total_amount,is_cod,is_cancelled
0,ORD202501023439,CUST181110,REST08622,DP05541,2025-01-01 12:00:00,471.62,35.44,30.56,466.74,N,N
1,ORD202501012051,CUST025572,REST02383,DP08091,2025-01-01 12:00:00,255.68,0.00,27.45,283.13,Y,N
2,ORD202501019281,CUST179306,REST14069,DP02021,2025-01-01 12:00:00,428.38,0.00,26.23,454.61,N,N
3,ORD202501000124,CUST191820,REST19745,DP13859,2025-01-01 12:00:00,260.81,0.00,32.75,293.56,N,N
4,ORD202501006518,CUST033760,REST12962,DP09615,2025-01-01 12:00:00,280.33,0.00,25.57,305.90,N,N


In [4]:
fact_orders_df.count()

order_id               149166
customer_id            149166
restaurant_id          149166
delivery_partner_id    143531
order_timestamp        149166
subtotal_amount        149166
discount_amount        149166
delivery_fee           149166
total_amount           149166
is_cod                 149166
is_cancelled           149166
dtype: int64

In [5]:
fact_orders_df.isnull().sum()

order_id                  0
customer_id               0
restaurant_id             0
delivery_partner_id    5635
order_timestamp           0
subtotal_amount           0
discount_amount           0
delivery_fee              0
total_amount              0
is_cod                    0
is_cancelled              0
dtype: int64

In [6]:
conn.execute("SELECT COUNT(*) FROM fact_orders WHERE delivery_partner_id IS NULL AND is_cancelled = 'Y'").fetchall()

[(5635,)]

*Rows where order is cancelled, only there the delivery partner ID is missing - that means it is not a data issue*

In [7]:
conn.execute("SELECT MIN(order_timestamp), MAX(order_timestamp) " \
"FROM fact_orders LIMIT 3;").fetch_df()

,min(order_timestamp),max(order_timestamp)
0,2025-01-01 12:00:00,2025-09-30 22:59:00


In [8]:
conn.execute("SELECT * FROM fact_order_items LIMIT 3;").fetch_df()

,order_id,item_id,menu_item_id,restaurant_id,quantity,unit_price,item_discount,line_total
0,ORD202501006518,ITEM001,MENU12962_3216,REST12962,2,48.31,0.0,96.62
1,ORD202501006518,ITEM002,MENU12962_1962,REST12962,3,61.24,0.0,183.71
2,ORD202501019281,ITEM001,MENU14069_0510,REST14069,2,87.19,0.0,174.38


In [9]:
conn.execute("SELECT * FROM fact_ratings LIMIT 3;").fetch_df()

,order_id,customer_id,restaurant_id,rating,review_text,review_timestamp,sentiment_score
0,ORD202501023439,CUST181110,REST08622,4.5,Super fast delivery,01-01-2025 15:00,0.75
1,ORD202501019281,CUST179306,REST14069,4.5,Great taste!,01-01-2025 15:00,0.75
2,ORD202501018036,CUST093042,REST13907,5.0,Super fast delivery,01-01-2025 14:03,1.00


In [10]:
conn.execute("SELECT * FROM fact_delivery_performance LIMIT 3;").fetch_df()

,order_id,actual_delivery_time_mins,expected_delivery_time_mins,distance_km
0,ORD202501023439,31,31,6.4
1,ORD202501012051,46,42,1.9
2,ORD202501019281,25,31,6.2


In [11]:
conn.execute("SELECT * FROM dim_customer LIMIT 3;").fetch_df()

,customer_id,signup_date,city,acquisition_channel
0,CUST000007,2025-03-21,Pune,Organic
1,CUST000008,2025-02-07,Kolkata,Referral
2,CUST000009,2025-02-25,Delhi,Paid


In [12]:
conn.execute("SELECT * FROM dim_delivery_partner_ LIMIT 3;").fetch_df()

,delivery_partner_id,partner_name,city,vehicle_type,employment_type,avg_rating,is_active
0,DP09615,Neha E,Bengaluru,Scooter,Full-time,3.77,Y
1,DP02021,Neha V,Bengaluru,Bike,Full-time,3.90,Y
2,DP05541,Vikas S,Delhi,Cycle,Part-time,3.33,Y


In [13]:
conn.execute("SELECT * FROM dim_menu_item LIMIT 3;").fetch_df()

,menu_item_id,restaurant_id,item_name,category,is_veg,price
0,MENU12962_3216,REST12962,Paneer Tikka Pizza,Pizza,Y,271.05
1,MENU12962_1962,REST12962,Pepperoni Pizza,Pizza,N,327.26
2,MENU14069_0510,REST14069,Egg Fried Rice,Fried Rice,N,200.01


In [14]:
conn.execute("SELECT * FROM dim_restaurant LIMIT 3;").fetch_df()

,restaurant_id,restaurant_name,city,cuisine_type,partner_type,avg_prep_time_min,is_active
0,REST12962,Flavours of Sweets Palace,Bengaluru,Chinese,Restaurant,26-40,N
1,REST14069,Royal Biryani Darbar,Ahmedabad,Fast Food,Restaurant,16-25,Y
2,REST08622,Spicy Wraps Point,Mumbai,North Indian,Restaurant,16-25,Y


# **Primary Analysis**

In [15]:

# 1. Compare total orders before and after the crisis
conn.execute("""
             WITH monthly_orders AS (
             SELECT 
                EXTRACT('MONTH' FROM order_timestamp) as month,
                CASE WHEN EXTRACT('MONTH' FROM order_timestamp) BETWEEN 1 AND 5 THEN 'Pre-Crisis (Jan-May 2025)'
                ELSE 'Crisis (Jun-Sept 2025)' END as time_frame,
                COUNT(order_id) as total_orders
             FROM fact_orders 
             GROUP BY EXTRACT('MONTH' FROM order_timestamp)
             )

             SELECT
               time_frame,
               SUM(total_orders) as total_orders,
               AVG(total_orders) as avg_orders_per_month
             FROM monthly_orders
             GROUP BY time_frame
             """).fetch_df()

,time_frame,total_orders,avg_orders_per_month
0,Pre-Crisis (Jan-May 2025),113806.0,22761.2
1,Crisis (Jun-Sept 2025),35360.0,8840.0


In [16]:

# 2. Cities with highest percentange decline in orders
conn.execute("""
             WITH city_level AS (
                  SELECT
                     r.city AS city,
                     COUNT(CASE WHEN EXTRACT(MONTH FROM o.order_timestamp) BETWEEN 1 AND 5 THEN 1 END) AS orders_pre,
                     COUNT(CASE WHEN EXTRACT(MONTH FROM o.order_timestamp) BETWEEN 6 AND 9 THEN 1 END) AS orders_crisis
                  FROM fact_orders o
                  JOIN dim_restaurant r
                     ON o.restaurant_id = r.restaurant_id
                  GROUP BY r.city
               ),

               percentage_calc AS (
                  SELECT
                     city,
                     orders_pre,
                     orders_crisis,
                     ROUND((orders_pre - orders_crisis) * 100.0 / NULLIF(orders_pre,0), 2) AS pct_decline
                  FROM city_level
               )

               SELECT
                  city,
                  pct_decline
               FROM percentage_calc
               WHERE orders_pre > 0
               ORDER BY pct_decline DESC
               LIMIT 5;
             """).fetch_df()

,city,pct_decline
0,Chennai,69.98
1,Kolkata,69.19
2,Bengaluru,69.17
3,Hyderabad,68.92
4,Ahmedabad,68.83


In [17]:

# 3. Restaurants with highest percentange decline in orders
conn.execute("""
             WITH restaurant_level AS (
                  SELECT
                     r.restaurant_name AS restaurant_name,
                     COUNT(CASE WHEN EXTRACT(MONTH FROM o.order_timestamp) BETWEEN 1 AND 5 THEN 1 END) AS orders_pre,
                     COUNT(CASE WHEN EXTRACT(MONTH FROM o.order_timestamp) BETWEEN 6 AND 9 THEN 1 END) AS orders_crisis
                  FROM fact_orders o
                  JOIN dim_restaurant r
                     ON o.restaurant_id = r.restaurant_id
                  GROUP BY r.restaurant_name
               ),

               percentage_calc AS (
                  SELECT
                     restaurant_name,
                     orders_pre,
                     orders_crisis,
                     ROUND((orders_pre - orders_crisis) * 100.0 / NULLIF(orders_pre,0), 2) AS pct_decline
                  FROM restaurant_level
                    WHERE orders_pre > 50
               )

               SELECT
                  restaurant_name,
                  pct_decline,
                  orders_pre,
                  orders_crisis
               FROM percentage_calc
               WHERE orders_pre > 0
               ORDER BY pct_decline DESC
               LIMIT 10
             """).fetch_df()

,restaurant_name,pct_decline,orders_pre,orders_crisis
0,Fresh Tandoor Delight,90.74,54,5
1,Flavours of Curry Cafe,90.57,53,5
2,Urban Kitchen Zone,85.07,67,10
3,Punjabi Curry Delight,84.48,58,9
4,Flavours of Tandoor Central,84.38,64,10
5,Hot & Crispy Mess Mahal,82.76,58,10
6,Sri Cafe Nest,82.69,52,9
7,Classic Sweets Heaven,82.46,57,10
8,Thindi Mane House Zone,82.35,51,9
9,Grand Cafe Clouds,82.26,62,11


In [18]:
# 4. Cancellation rate trend pre vs crisis, whcih cities are most affected

conn.execute("""
WITH city_level AS (
SELECT
   r.city AS city,

   /* Pre-crisis */
   SUM(CASE WHEN EXTRACT(MONTH FROM o.order_timestamp) BETWEEN 1 AND 5
            AND o.is_cancelled='Y' THEN 1 ELSE 0 END) AS cancels_pre,

   COUNT(CASE WHEN EXTRACT(MONTH FROM o.order_timestamp) BETWEEN 1 AND 5
              THEN 1 END) AS orders_pre,

   /* Crisis */
   SUM(CASE WHEN EXTRACT(MONTH FROM o.order_timestamp) BETWEEN 6 AND 9
            AND o.is_cancelled='Y' THEN 1 ELSE 0 END) AS cancels_crisis,

   COUNT(CASE WHEN EXTRACT(MONTH FROM o.order_timestamp) BETWEEN 6 AND 9
              THEN 1 END) AS orders_crisis

FROM fact_orders o
JOIN dim_restaurant r
  ON o.restaurant_id = r.restaurant_id

GROUP BY r.city
),

rate_calc AS (
SELECT
   city,

   ROUND(100.0 * cancels_pre / NULLIF(orders_pre,0),2) AS cancellation_rate_pre,

   ROUND(100.0 * cancels_crisis / NULLIF(orders_pre,0),2) AS cancellation_rate_crisis,

   ROUND(
     100.0 * cancels_crisis / NULLIF(orders_crisis,0)
     -
     100.0 * cancels_pre / NULLIF(orders_pre,0)
   ,2) AS pp_increase
FROM city_level
)

SELECT
  city,
  cancellation_rate_pre,
  cancellation_rate_crisis,
  pp_increase
FROM rate_calc
ORDER BY pp_increase DESC
LIMIT 10;

             """).fetch_df()

,city,cancellation_rate_pre,cancellation_rate_crisis,pp_increase
0,Ahmedabad,6.08,4.06,6.95
1,Mumbai,5.87,3.87,6.48
2,Chennai,6.07,3.66,6.12
3,Kolkata,5.86,3.69,6.10
4,Hyderabad,6.10,3.76,6.00
5,Pune,6.00,3.73,5.62
6,Bengaluru,6.17,3.63,5.59
7,Delhi,6.16,3.48,4.90


In [19]:
# 5. Avg delivery times across phases - did it worsen in crisis period
conn.execute("""
            WITH order_level_delay AS (
            SELECT 
                o.order_id,
                EXTRACT(MONTH FROM o.order_timestamp) AS month,
                CASE WHEN EXTRACT(MONTH FROM o.order_timestamp) BETWEEN 1 AND 5 THEN 'Pre-Crisis'
                ELSE 'Crisis' END AS phase,
                d.actual_delivery_time_mins,
                d.expected_delivery_time_mins,
                d.actual_delivery_time_mins - d.expected_delivery_time_mins AS delay
            FROM fact_delivery_performance d 
            JOIN fact_orders o
            ON d.order_id = o.order_id
                WHERE o.is_cancelled != 'Y'
             )

             SELECT
                phase,
                AVG(delay) as avg_delay_mins,
                AVG(actual_delivery_time_mins) AS avg_delivery_time
             FROM order_level_delay
                GROUP BY phase
             """).fetch_df()

,phase,avg_delay_mins,avg_delivery_time
0,Crisis,17.598227,60.126453
1,Pre-Crisis,2.020175,39.522551


In [27]:
# 6. Avg customer rating MoM trend
conn.execute("""
            WITH ratings_per_order AS (
            SELECT 
                EXTRACT(MONTH FROM o.order_timestamp) AS month,
                CASE WHEN EXTRACT(MONTH FROM o.order_timestamp) BETWEEN 1 AND 5 THEN 'Pre-Crisis'
                ELSE 'Crisis' END AS phase,
                AVG(r.rating) AS avg_rating
            FROM fact_ratings r
            JOIN fact_orders o
            ON r.order_id = o.order_id
                WHERE o.is_cancelled != 'Y'
            GROUP BY EXTRACT(MONTH FROM o.order_timestamp)
             )

             SELECT
                month,
                avg_rating,
                (1.0 * ((LAG(avg_rating) OVER(ORDER BY month) - avg_rating) / LAG(avg_rating) OVER(ORDER BY month) )) * -100.00 AS pct_change
             FROM ratings_per_order
                
             """).fetch_df()

,month,avg_rating,pct_change
0,1,4.576851,NaN
1,2,4.398034,-3.906991
2,3,4.739867,7.772406
3,4,4.292625,-9.435749
4,5,4.490320,4.605457
5,6,2.584811,-42.435936
6,7,2.690036,4.070909
7,8,2.400354,-10.768718
8,9,2.305317,-3.959291


In [31]:
# 7. Most frequently occuring negative keywords in reviews during crisis period

conn.execute("""
            WITH crisis_reviews AS (
                SELECT
                f.review_text,
                f.sentiment_score,
                r.city
                FROM fact_ratings f
                JOIN fact_orders o
                ON f.order_id = o.order_id
                JOIN dim_restaurant r
                ON f.restaurant_id = r.restaurant_id
                WHERE EXTRACT(MONTH FROM o.order_timestamp) BETWEEN 6 AND 9
                AND f.sentiment_score < 0
                ),

                tokens AS (
                SELECT
                city,
                UNNEST(
                    string_split(
                    lower(regexp_replace(review_text,'[^a-zA-Z ]','')),
                    ' ')
                ) AS word
                FROM crisis_reviews
                ),

                clean_tokens AS (
                SELECT *
                FROM tokens
                WHERE length(word) > 2 AND word NOT IN ('the','and','for','with','that','this','was')
                )

                SELECT
                word,
                COUNT(*) AS frequency
                FROM clean_tokens
                GROUP BY word
                ORDER BY frequency DESC;

             """).fetch_df()

,word,frequency
0,food,4941
1,not,3823
2,quality,2226
3,issue,2073
4,packaging,1627
5,good,1596
6,safety,1076
7,taste,972
8,bad,972
9,served,971


In [44]:
# 8. Estimate revenue loss: pre-crisis vs crisis

conn.execute("""
            WITH monthly_revenue AS (
                SELECT
                EXTRACT(MONTH FROM order_timestamp) AS month,

                CASE 
                    WHEN EXTRACT(MONTH FROM order_timestamp) BETWEEN 1 AND 5 THEN 'Pre-Crisis'
                    WHEN EXTRACT(MONTH FROM order_timestamp) BETWEEN 6 AND 9 THEN 'Crisis'
                END AS phase,

                SUM(total_amount) AS revenue_month

                FROM fact_orders
                WHERE is_cancelled='N'

                GROUP BY month, phase
                )

                SELECT
                phase,
                ROUND(AVG(revenue_month),0) AS avg_monthly_revenue
                FROM monthly_revenue
                GROUP BY phase;
             """).fetch_df()

,phase,avg_monthly_revenue
0,Pre-Crisis,7524193.0
1,Crisis,2735038.0


In [ ]:
# 9. Customers with 5 or more orders before crisis - how many stopped ordering during the crisis 
# and from them how many had avg rating above 4.5

conn.execute("""
                WITH pre_orders AS (
                SELECT
                  customer_id,
                  COUNT(*) AS orders_pre
                FROM fact_orders
                WHERE is_cancelled='N'
                AND EXTRACT(MONTH FROM order_timestamp) BETWEEN 1 AND 5
                GROUP BY customer_id
                ),

                crisis_orders AS (
                SELECT
                  customer_id,
                  COUNT(*) AS orders_crisis
                FROM fact_orders
                WHERE is_cancelled='N'
                AND EXTRACT(MONTH FROM order_timestamp) BETWEEN 6 AND 9
                GROUP BY customer_id
                ),

                target_customers AS (
                SELECT 
                  p.customer_id
                FROM pre_orders p
                LEFT JOIN crisis_orders c
                ON p.customer_id = c.customer_id
                WHERE p.orders_pre >= 5
                AND COALESCE(c.orders_crisis,0) = 0
                ),

                stopped_with_rating AS (
                SELECT
                t.customer_id,
                AVG(r.rating) AS avg_rating
                FROM target_customers t
                JOIN fact_ratings r
                ON t.customer_id = r.customer_id
                GROUP BY t.customer_id
                )

                SELECT
                COUNT(*) AS stopped_customers,
                COUNT(CASE WHEN avg_rating > 4.5 THEN 1 END) AS high_rating_stopped
                FROM stopped_with_rating;

             """).fetch_df()

,stopped_customers,high_rating_stopped
0,41,22


In [56]:
# 10. Which high-value customers (top 5% by total spend before crisis) showed largest drop 
# in order freq and ratings during the crisis
# What are the common patterns - location, cuisine preference, delivery delay

conn.execute("""
                WITH pre_total_spend AS (
                SELECT
                  customer_id,
                  SUM(total_amount) as total_spend
                FROM fact_orders
                WHERE is_cancelled='N'
                AND EXTRACT(MONTH FROM order_timestamp) BETWEEN 1 AND 5
                GROUP BY customer_id
                ),

                crisis_orders AS (
                SELECT
                  customer_id,
                  COUNT(*) AS orders_crisis
                FROM fact_orders
                WHERE is_cancelled='N'
                AND EXTRACT(MONTH FROM order_timestamp) BETWEEN 6 AND 9
                GROUP BY customer_id
                )

                SELECT
                    *,
                    PERCENT_RANK() OVER(ORDER BY total_spend DESC) as percent_rank
                FROM pre_total_spend
                    ORDER BY 2 DESC

             """).fetch_df()

,customer_id,total_spend,percent_rank
0,CUST077722,2715.18,0.000000
1,CUST179266,2404.06,0.000012
2,CUST125990,2403.16,0.000024
3,CUST163628,2397.58,0.000036
4,CUST165515,2349.96,0.000048
...,...,...,...
82857,CUST078838,117.74,0.999952
82858,CUST069694,117.17,0.999964
82859,CUST030496,116.98,0.999976
82860,CUST187118,116.53,0.999988
